<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

# Notebook Set E

---

## 03 - Complex Representation of QAM <a class="anchor" id="complex_QAM"></a>
The QAM model we have looked at so far has used two real signals as the input waveform. A complex representation is also possible. This model is functionally identical to the previous model, however, it is simpler to implement as only a single mixer is required. In this notebook, we focus on demonstrating that the complex representation is equivalent to the original model, without going into the underlying maths in great detail.

## Table of Contents
* [1. Introduction](#introduction)
* [2. Complex Modulation](#complex_QAM_mod)
* [3. Complex Demodulation](#complex_QAM_demod)
* [4. Conclusion](#conclusion)
    
## Revision
* **v1.0** | 22/12/22 | *First Revision*

---


## 1. Introduction <a class="anchor" id="introduction"></a>
Consider a quadrature transmitter described using complex notation, as shown in Figure 1.

<figure>
<img src='./images/complex_modulation.svg' height='60%' width='60%'/>
    <figcaption><b>Figure 1: Complex modulation architecture.</b></figcaption>
</figure>

Where:

$$
g(t) = g_{1}(t) + jg_{2}(t),
$$

and $g_{1}$ and $g_{2}$ are the same signals used in the QAM modulation example.

$$
g_{1}(t) = A_{1} \cos(2 \pi f_{b1} t)
$$

$$
g_{2}(t) = A_{2} \cos(2 \pi f_{b2} t)
$$

For this notebook, we will use NumPy for computation and MatplotLib for visualisation of our waveforms. Let us begin by importing these libraries now.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 2. Complex Modulation <a class="anchor" id="complex_QAM_mod"></a>
First off, we define the two "information" signals, as seen in the previous notebook. The signals we use here to represent information are simple sine waves, which are not really carrying information. However, they are useful for the purposes of our example. We use the term "In Phase" for our real component and "Quadrature" for the imaginary component.

In [ ]:
# Set basic params
fs = 4096e6 # sample rate
fb = 64e6 # frequency of baseband signal
fb2 = 32e6 # frequency of second baseband signal
A1 = 2 # baseband signal amplitude
A2 = 1 # second baseband signal amplitude
N_fft = 2048 # fft size

t = np.arange(N_fft)/fs #time scale

In [ ]:
# Define two information signals
g1 = A1*np.cos(2*np.pi*fb*t)
g2 = A2*np.cos(2*np.pi*fb2*t)

We now define the complex signal $g(t)$ by summing together $g_{1}(t)$ and a complex signal, $g_{2}(t)$.

In [ ]:
# Define complex input signal using real g1(t) and imag. g2(t)
g_complex = g1 + 1j*(g2)

In [ ]:
# Calculate FFT
g_complex_fft_result = np.fft.fft(g_complex, N_fft)

# Get the corresponding frequencies, that depend on N_fft and Fs - freq. domain x axis
freqs = np.fft.fftfreq(N_fft,1/fs)

If we plot this complex information signal in the time domain, we can observe that the information signal matches $g_{1}$ and $g_{2}$, which was previously shown in the QAM example. 

We also show the spectra of the baseband signals as both a one sided and two sided frequency plot. Up to this point, we have only explored one sided plots, as our signals have been real, and exhibit symmetrical positive and negative frequencies. In this notebook, we will explore complex signals, which are non-symmetrical and contain information in both positive and negative frequencies.

For the two sided frequency plot below, it was worth noting the symmetry that exists in the signal.  

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(20,4))
axs[0].plot(g_complex.real[:200])
axs[0].plot(g_complex.imag[:200])
axs[0].set_title('Time Domain Complex Information Signal')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')
axs[0].legend(('I signal Component', 'Q Signal Component'))

axs[1].plot(freqs[:int(N_fft/2)]/1e6, np.abs(g_complex_fft_result[:int(N_fft/2)]))
axs[1].set_title('One Sided FFT plot of the Complex Information Signal')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

axs[2].plot(freqs[:int(N_fft)]/1e6, np.abs(g_complex_fft_result[:int(N_fft)]))
axs[2].set_title('Two Sided FFT plot of the Complex Information Signal')
axs[2].set_ylabel('Magnitude')
axs[2].set_xlabel('Frequency, MHz')
fig.tight_layout()

This complex information signal can be modulated using a complex exponential at a 'frequency' of $f_{c}$ Hz. The equation below shows Euler's formula, which describes the relationship between a complex exponential and associated trigonometric functions.

$$
e^{j 2 \pi f_{c} t} = \cos (2 \pi f_{c} t) + j \sin (2 \pi f_{c} t).
$$

In [ ]:
fc = 1400e6 # Carrier Frequency

# Define complex exponential used for modulation
c_complex = np.exp(1j*2*np.pi*fc*t)

In [ ]:
# Find FFT of complex exponential
c_complex_fft_result = np.fft.fft(c_complex, N_fft)

Upon running the code cell below, we can see that there are no negative components in the frequency domain. This exponential carrier is simply a signal at 900 MHz and does not contain any symmetry in the frequency domain.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,4))
axs[0].plot(c_complex.real[:100])
axs[0].plot(c_complex.imag[:100])
axs[0].set_title('Time Domain Complex Carrier')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')
axs[0].legend(('I Component', 'Q Component'))

axs[1].plot(freqs[:int(N_fft)]/1e6, np.abs(c_complex_fft_result[:int(N_fft)]))
axs[1].set_title('Two Sided FFT plot of the Complex Carrier')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

This modulator creates the following signal, achieved by multiplying $g(t)$ with $e^{j 2 \pi f_{c} t}$, as below.

$$
v(t) = g(t)e^{j 2 \pi f_{c} t} = [g_{1}(t) + jg_{2}(t)]e^{j 2 \pi f_{c} t}
$$

We know that Euler's Formula relates to the trigonometric world such that

$$
e^{j 2 \pi f_{c} t} = \cos(2 \pi f_{c} t) + j \sin(2 \pi f_{c} t).
$$

Therefore,

$$
v(t) = g(t)e^{j 2 \pi f_{c} t} = [g_{1}(t) + jg_{2}(t)][\cos(2 \pi f_{c} t) + j \sin(2 \pi f_{c} t)],
$$

$$
v(t) = g_{1}(t)\cos(2 \pi f_{c} t) + jg_{2}(t)\cos(2 \pi f_{c} t) + jg_{1}(t)\sin(2 \pi f_{c} t) - g_{2}(t)\sin(2 \pi f_{c} t).
$$

We can now extract the real and imaginary components, which gives

$$
v(t) = [g_{1}(t) \cos (2 \pi f_{c} t) - g_{2}(t) \sin (2 \pi f_{c} t)] + j [g_{1}(t) \sin (2 \pi f_{c} t) + g_{2}(t) \cos(2 \pi f_{c} t)].
$$

We can do this sum in Python using a simple multiplication of the complex information signal with the complex exponential.

In [ ]:
# Find complex modulated signal v(t)
v = g_complex*c_complex

In [ ]:
# Find FFT of real modulated signal
v_fft_result = np.fft.fft(v, N_fft)

As we might expect from the exponential carrier, modulation results in positive frequency components only. Therefore, we are able see frequency components at $f_{c} - f_{b}$ and $f_{c} + f_{b}$ for each signal.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,4))
axs[0].plot(v.real[:200])
axs[0].plot(v.imag[:200])
axs[0].set_title('Time Domain Complex Carrier Signal')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')
axs[0].legend(('I signal Component', 'Q Signal Component'))

axs[1].plot(freqs[:int(N_fft)]/1e6, np.abs(v_fft_result[:int(N_fft)]))
axs[1].set_title('Two Sided FFT plot of the Complex Modulated Signal')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

Information about both $g_{1}(t)$ and $g_{2}(t)$ is contained in the real portion of the signal and as a result the imaginary part can be removed using a 'Real' operator, prior to transmission. This operation gives us,

$$
y(t) = g_{1}(t) \cos (2 \pi f_{c} t) - g_{2}(t) \sin (2 \pi f_{c} t).
$$

In [ ]:
# Find Real modulated signal
y = v.real

In [ ]:
# Find FFT of real modulated signal
y_fft_result = np.fft.fft(y, N_fft)

We can observe from the plots below that the modulated signal is identical to that produced by the QAM model presented in the previous notebook. Note that a one-sided spectrum is shown here, as the signal is real.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,4))
axs[0].plot(y[:200])
axs[0].set_title('Time Domain Real Modulated Signal')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')

axs[1].plot(freqs[:int(N_fft/2)]/1e6, np.abs(y_fft_result[:int(N_fft/2)]))
axs[1].set_title('One Sided FFT plot of the Real Modulated Signal')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

## 3. Complex Demodulation <a class="anchor" id="complex_QAM_demod"></a>

QAM Demodulation can also be expressed using complex notation, shown in [Figure 2](#fig-2).:

<a class="anchor" id="fig-2"></a>
<figure>
<img src='./images/complex_demodulation.svg' height='60%' width='60%'/>
    <figcaption><b>Figure 2: Complex demodulation architecture.</b></figcaption>
</figure>

We aim to demodulate the received signal, or 'shift' the spectrum to the left. Therefore our local oscillator should be the negative of the transmitter's oscillator.

In [ ]:
# Define local exponential carrier at the receiver
complex_demod_carrier = np.exp(-1j*2*np.pi*fc*t)

In [ ]:
# Find FFT of real modulated signal
complex_demod_carrier_fft_result = np.fft.fft(complex_demod_carrier, N_fft)

We can confirm in the frequency domain that the local oscillator is negative as we see a single frequency component at -900 MHz. 

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,4))
axs[0].plot(complex_demod_carrier.real[:100])
axs[0].plot(complex_demod_carrier.imag[:100])
axs[0].set_title('Time Domain Complex Carrier')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')
axs[0].legend(('I Component', 'Q Component'))

axs[1].plot(freqs[:int(N_fft)]/1e6, np.abs(complex_demod_carrier_fft_result[:int(N_fft)]))
axs[1].set_title('Two Sided FFT plot of the Complex Carrier')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

We multiply the received signal $y_{t}$ with the local oscillator, which gives

$$
x(t) = y(t)e^{-j 2 \pi f_{c} t} = [g_{1}(t) \cos (2 \pi f_{c} t) - g_{2}(t) \sin (2 \pi f_{c} t)]e^{-j 2 \pi f_{c} t},
$$

$$
x(t) = [g_{1}(t) \cos (2 \pi f_{c} t) - g_{2}(t) \sin (2 \pi f_{c} t)] [\cos (2 \pi f_{c} t) - j\sin (2 \pi f_{c} t)].
$$

Expanding the square brackets above and reordering gives

$$
x(t) = g_{1}(t) \cos ^2(2 \pi f_{c} t) + j\sin ^2 (2 \pi f_{c} t) - jg_{1}(t)\cos (2 \pi f_{c} t)\sin (2 \pi f_{c} t) - g_{2}(t)\sin (2 \pi f_{c} t)\cos (2 \pi f_{c} t).
$$

Using trigonometric identities we can reduce this to

$$
x(t) = 0.5[g_{1}(t) + jg_{2}] + 0.5(g_{1}(t) \cos(4 \pi f_{c} t) - j0.5(g_{2}(t) \cos(4 \pi f_{c} t) - j0.5(g_{1}(t) \sin(4 \pi f_{c} t) - 0.5(g_{2}(t) \sin(4 \pi f_{c} t).
$$

In [ ]:
# Demodulate by multiplying y(t) by the exponential
x = y*complex_demod_carrier

In [ ]:
# Find FFT of demodulated signals
x_fft_result = np.fft.fft(x, N_fft)

In the frequency domain, we can observe that the desired signal has been shifted and is now centred around 0 Hz, i.e. it has been successfully demodulated. However, a second copy of the signal can also be seen, centred around $-2 f_{c}$.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,4))
axs[0].plot(x.real[:200])
axs[0].plot(x.imag[:200])
axs[0].set_title('Time Domain Complex Demodulated Signal')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')
axs[0].legend(('I Demodulated Signal', 'Q Demodulated Signal'))

axs[1].plot(freqs[:int(N_fft)]/1e6, np.abs(x_fft_result[:int(N_fft)]))
axs[1].set_title('Two Sided FFT plot of the Complex Demodulated Signal')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

In a similar way to the previous example, we can apply a low pass filter to this signal to remove the unwanted components. The lowpass filter is symmetrical and so it will remove the unwanted components centred at $-2 f_{c}$, which gives

$$
z(t) = 0.5[g_{1}(t) + jg_{2}].
$$

In other words, the original "information" signal that was transmitted has been retrieved, but it is scaled by 0.5.

In [ ]:
# Define lowpass filter
f_cutoff = 0.1 # Cutoff frequency as a fraction of the sampling rate
b = 0.08  # Transition band, as a fraction of the sampling rate (in (0, 0.5)).

N = int(np.ceil((4 / b)))
if not N % 2: N += 1  # N is odd.
n = np.arange(N)

h = np.sinc(2 * f_cutoff * (n - (N - 1) / 2)) # Compute sinc filter.
w = np.blackman(N) # Compute Blackman window.
h = h * w # Multiply sinc filter by window.
h = h / np.sum(h) # Normalize to get unity gain.

In [ ]:
# Apply filter to demodulated signal
z = np.convolve(x, h)

In [ ]:
# Find FFT of filtered signal
z_fft_result = np.fft.fft(z, N_fft)

Upon running the code cell below, we see that the reconstructed signal matches the input signal in both time and frequency domains.

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(20,4))
axs[0].plot(z.real[:200])
axs[0].plot(z.imag[:200])
axs[0].set_title('Time Domain Lowpass \nFiltered Complex Demodulated Signal')
axs[0].set_xlabel('Samples')
axs[0].set_ylabel('Amplitude')
axs[0].legend(('I Signal Component', 'Q Signal Component'))

axs[1].plot(freqs[:int(N_fft/2)]/1e6, np.abs(z_fft_result[:int(N_fft/2)]))
axs[1].set_title('One Sided FFT plot of the Lowpass \nFiltered Complex Demodulated Signal')
axs[1].set_ylabel('Magnitude')
axs[1].set_xlabel('Frequency, MHz')
fig.tight_layout()

axs[2].plot(freqs[:int(N_fft)]/1e6, np.abs(z_fft_result[:int(N_fft)]))
axs[2].set_title('Two Sided FFT plot of the Lowpass\n Filtered Complex Demodulated Signal')
axs[2].set_ylabel('Magnitude')
axs[2].set_xlabel('Frequency, MHz')
fig.tight_layout()

We can therefore confirm that the complex modulation and demodulation presented has been successful, and moreover it has produced the same results as the modulation / demodulation from the previous notebook.

## 4. Conclusion <a class="anchor" id="conclusion"></a>
In this notebook, we have explored a complex QAM architecture, and demonstrated how we can exploit the properties of a complex exponential mixer to modulate and demodulate a complex information signal. This method produces exactly equivalent results to the real QAM modulation and demodulation presented in the previous notebook, but it can be more concisely represented, both diagrammatically and mathematically. Working with the complex mathematical notation is also more tractable.

The next notebook will explore the topic of frequency planning in wireless communication systems.

---

[⬅️ Previous Notebook](02_qam_modulation.ipynb) || [Next Notebook 🚀](../notebook_F/01_frequency_planning.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---